In [60]:
#dependencies
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
import requests
from config import api_key
import json
from pprint import pprint

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [27]:
url='https://www.the-numbers.com/movie/budgets/all'
browser.visit(url)

In [28]:
movie_df=[]

In [29]:
#scrapes top 500 movie production budgets
page_list=['101-','201-','301-','401-','501-']
for page in page_list:
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    table=soup.find_all('table')[0]
    df=pd.read_html(str(table))
    df=df[0]
    df=df.drop(columns='Unnamed: 0')
    movie_df.append(df)
    browser.click_link_by_partial_text(page)

C:\Users\nickc\anaconda\Anaconda3\envs\pythondata\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [30]:
#appends movie data into one dataframe 
movies_df=movie_df[0].append(movie_df[1]).append(movie_df[2]).append(movie_df[3]).append(movie_df[4]).reset_index().drop(columns='index')

In [31]:
movies_df

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963"
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,068,223,624"
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754"
...,...,...,...,...,...
495,"Jul 7, 2004",King Arthur,"$90,000,000","$51,877,963","$203,877,963"
496,"Apr 25, 1997",Volcano,"$90,000,000","$47,546,796","$120,100,000"
497,"Mar 26, 1997",The Devil's Own,"$90,000,000","$42,885,593","$140,900,000"
498,"Jul 19, 2002",K-19: The Widowmaker,"$90,000,000","$35,168,966","$65,716,126"


In [32]:
movies_df.to_csv('movies_data.csv')

In [118]:
#OMDB url
url=f'http://www.omdbapi.com/?t='
#empty lists to be appended later
imdb_rating_list=[]
metascore_list=[]
rotten_tomatoes_score_list=[]
rated_list=[]
#OMDB api call for ratings
for movie in movies_df['Movie']:
    print(movie)
    response=requests.get(url+movie+'&apikey='+api_key)
    data=response.json()
    try:
        
        try:
            imdb_rating=data['Ratings'][0]['Value']
        except:
            imdb_rating='NaN'
        metascore=data['Metascore']
        try:
            rotten_tomatoes=data['Ratings'][1]['Value']
        except IndexError:
            rotten_tomatoes='NaN'
        rated=data['Rated']
    except:
        imdb_rating='NaN'
        rotten_tomatoes='NaN'
        metascore='NaN'
        rated='NaN'
    metascore_list.append(metascore)
    imdb_rating_list.append(imdb_rating)
    rotten_tomatoes_score_list.append(rotten_tomatoes)
    rated_list.append(rated)
    

Avengers: Endgame
Pirates of the Caribbean: On Stranger Tides
Avengers: Age of Ultron
Star Wars Ep. VII: The Force Awakens
Avengers: Infinity War
Pirates of the Caribbean: At World’s End
Justice League
Spectre
Star Wars: The Rise of Skywalker
Solo: A Star Wars Story
John Carter
Batman v Superman: Dawn of Justice
The Lion King
Tangled
Spider-Man 3
Captain America: Civil War
Harry Potter and the Half-Blood Prince
The Hobbit: The Desolation of Smaug
The Hobbit: The Battle of the Five Armies
The Fate of the Furious
No Time to Die
Avatar
Superman Returns
The Dark Knight Rises
Pirates of the Caribbean: Dead Men Tell No Tales
Quantum of Solace
The Avengers
Pirates of the Caribbean: Dead Man’s Chest
Man of Steel
The Chronicles of Narnia: Prince Caspian
The Lone Ranger
Tenet
The Amazing Spider-Man
Battleship
Transformers: The Last Knight
Jurassic World
Men in Black 3
Transformers: Revenge of the Fallen
Transformers: Age of Extinction
X-Men: The Last Stand
Robin Hood
King Kong
The Golden Compass

Ford v. Ferrari
Gangs of New York
Alien: Covenant
Cinderella
Ice Age: Continental Drift
Black Hawk Down
Sex and the City 2
Percy Jackson & the Olympians: The Lightning Thief
The Campaign
Unstoppable
The Fifth Element
The Road to El Dorado
The Lovely Bones
Ben-Hur
Seventh Son
Finding Nemo
The Lord of the Rings: The Return of the King
The Lord of the Rings: The Two Towers
Lara Croft: Tomb Raider
Kingsman: The Secret Service
Jurassic Park III
Rise of the Planet of the Apes
The Spiderwick Chronicles
The Incredibles
A Good Day to Die Hard
The Alamo
Cutthroat Island
The Secret Life of Walter Mitty
Jumanji: Welcome to the Jungle
Men in Black
Toy Story 2
Rush Hour 2
Sherlock Holmes
Ice Age: Dawn of the Dinosaurs
Spider-Man: Into The Spider-Verse 3D
Bad Boys For Life
What Lies Beneath
Rio
Once Upon a Time…in Hollywood
The Departed
Charlie's Angels
Mulan
Tropic Thunder
The Girl with the Dragon Tattoo
Contact
You Don't Mess With the Zohan
Die Hard: With a Vengeance
The Magnificent Seven
Alvin and

In [119]:
#adding lists to movies DataFrame
movies_df['Rated']=rated_list
movies_df['IMDB_Rating']=imdb_rating_list
movies_df['Metascore']=metascore_list
movies_df['Rotten Tomatoes']=rotten_tomatoes_score_list
movies_df.head()

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564",PG-13,8.5/10,78,94%
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875",PG-13,6.6/10,45,33%
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963",PG-13,7.3/10,66,75%
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,068,223,624",NaN,NaN,NaN,NaN
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754",PG-13,8.5/10,68,85%


In [140]:
#droping all rows with NaN values
movies_NaN=movies_df[movies_df['Rotten Tomatoes']=='NaN'].index
movies_df=movies_df.drop(movies_NaN)
movies_df=movies_df.reset_index()
movies_df=movies_df.drop(columns=['index'])
movies_df


,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross,Rated,IMDB_Rating,Metascore,Rotten Tomatoes
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564",PG-13,8.5/10,78,94%
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875",PG-13,6.6/10,45,33%
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,403,013,963",PG-13,7.3/10,66,75%
3,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754",PG-13,8.5/10,68,85%
4,"May 24, 2007",Pirates of the Caribbean: At World’s End,"$300,000,000","$309,420,425","$963,420,425",PG-13,7.1/10,50,44%
...,...,...,...,...,...,...,...,...,...
459,"Jul 7, 2004",King Arthur,"$90,000,000","$51,877,963","$203,877,963",PG-13,6.3/10,46,31%
460,"Apr 25, 1997",Volcano,"$90,000,000","$47,546,796","$120,100,000",PG-13,5.5/10,55,50%
461,"Mar 26, 1997",The Devil's Own,"$90,000,000","$42,885,593","$140,900,000",R,6.2/10,53,34%
462,"Jul 19, 2002",K-19: The Widowmaker,"$90,000,000","$35,168,966","$65,716,126",PG-13,6.7/10,58,61%


In [142]:
conn='mongodb://localhost:27017'
client=pymongo.MongoClient(conn)

In [154]:
db=client.movies_db
db.command('dropDatabase')
collection=db.movies


In [155]:
for i in movies_df.index:
    movie_dict=movies_df.iloc[i].to_dict()
    collection.insert_one(movie_dict)